# Explainability for Classifiers: GradCAM, OcclusionSensitivity et al.

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/mtwenzel/teaching/blob/master/01 Explainability using tf_explain.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/mtwenzel/teaching/blob/master/01 Explainability using tf_explain.ipynb
"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>
<br/><br/><br/>

This notebook shows for MNIST and for a medical example (Parkinson SPECT classification) how different visualization methods compare.

The code inherits from the 'tf_explain' original authors' example code and adapts it to the Parkinson example.

Use as a basis for own experiments.

# Preparations

Install TensorFlow 2.0.0 rc0 and TFP 0.8.0 rc0 below, if not running locally.

In [0]:
#@title Remove Tensorboard logs of previous runs
#@markdown Don't execute if you don't want to loose logs. 
%rm -rf logs/

In [0]:
#@title Install TensorFlow { display-mode: "form" }
TF_Installation = 'TF2 RC0 (GPU)' #@param ['TF2 Nightly (GPU)', 'TF2 RC0 (GPU)', 'TF2 Stable (GPU)', 'TF1 Nightly (GPU)', 'TF1 Stable (GPU)','System']
# added 2.0.0-rc0
if TF_Installation == 'TF2 Nightly (GPU)':
  !pip install -q --upgrade tf-nightly-gpu-2.0-preview
  print('Installation of `tf-nightly-gpu-2.0-preview` complete.')
elif TF_Installation == 'TF2 RC0 (GPU)':
  !pip install -q --upgrade tensorflow-gpu==2.0.0-rc0
  print('Installation of `tensorflow-gpu==2.0.0-rc0` complete. Use with tensorflow_probability=0.8.0-rc0')
elif TF_Installation == 'TF2 Stable (GPU)':
  !pip install -q --upgrade tensorflow-gpu==2.0.0-alpha0
  print('Installation of `tensorflow-gpu==2.0.0-alpha0` complete.')
elif TF_Installation == 'TF1 Nightly (GPU)':
  !pip install -q --upgrade tf-nightly-gpu
  print('Installation of `tf-nightly-gpu` complete.')
elif TF_Installation == 'TF1 Stable (GPU)':
  !pip install -q --upgrade tensorflow-gpu
  print('Installation of `tensorflow-gpu` complete.')
elif TF_Installation == 'System':
  pass
else:
  raise ValueError('Selection Error: Please select a valid '
                   'installation option.')

In [0]:
#@title Install tensorflow_probability { display-mode: "form" }
TFP_Installation = "0.8.0-rc0" #@param ["0.8.0-rc0", "Nightly", "Stable", "System"]

if TFP_Installation == "Nightly":
  !pip install -q tfp-nightly
  print("Installation of `tfp-nightly` complete.")
elif TFP_Installation == "0.8.0-rc0":
  !pip install -q --upgrade tensorflow-probability==0.8.0-rc0
  print("Installation of `tensorflow-probability` complete.")
elif TFP_Installation == "Stable":
  !pip install -q --upgrade tensorflow-probability
  print("Installation of `tensorflow-probability` complete.")
elif TFP_Installation == "System":
  pass
else:
  raise ValueError("Selection Error: Please select a valid "
                   "installation option.")

In [0]:
#@title Check GPU availability and TF version. 
import tensorflow as tf

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    print('GPU device not found')
else:
    print('Found GPU at: {}'.format(device_name))
print(tf.__version__ )# Has to be 2.0 for this notebook to work...

In [0]:
#@title Install required python packages and utility packages.
#@markdown This installs tf_explain and RAdam, a performant optimizer. Also downloads and imports two helper python sources.
# https://github.com/sicara/tf-explain
try:
  import tf_explain as tfx
except:
  !pip install tf_explain
  import tf_explain as tfx

#@markdown Also install talos for easy hyperparameter tuning.
try:
  import talos
except:
  !pip install talos
  import talos

# https://github.com/CyberZHG/keras-radam
try:
  from keras_radam.training import RAdamOptimizer # for TF
except:
  !pip install keras-rectified-adam
  from keras_radam.training import RAdamOptimizer
    
from urllib.request import urlopen
try:
  import utilities
except:
  url = 'https://github.com/mtwenzel/utilities/raw/master/utilities.py'
  resp = urlopen(url)
  temp = open("utilities.py", "wb")
  temp.write(resp.read())
  temp.close()
  import utilities

try:
  import data_loaders
except:
  url = 'https://github.com/mtwenzel/utilities/raw/master/data_loaders.py'
  resp = urlopen(url)
  temp = open("data_loaders.py", "wb")
  temp.write(resp.read())
  temp.close()
  import data_loaders

In [0]:
#@title Further imports and setup {display-mode:"form"}
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Input,Conv2D,Dense,GlobalAveragePooling2D,concatenate,Flatten, MaxPooling2D, BatchNormalization, Dropout, SpatialDropout2D
from tensorflow.keras.applications import InceptionV3,DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import seaborn as sns
sns.set()
%matplotlib inline

In [0]:
#@title Prepare the data. {display-mode:'form'}
TARGET_SIZE = (96,96) # Square images because of visualization library...
paths_dict = {'train': './data/PPMI-classification/all_2d_train',
             'val': './data/PPMI-classification/all_2d_val',
             'test': './data/PPMI-classification/all_2d_val'}

train_generator, val_generator, test_generator = data_loaders.provide_PPMI_dataset(paths_dict, target_size=TARGET_SIZE)
%rm PPMI-classification.zip

# Model Definition and Training

In [0]:
#@title This is a performant model definition. It is hard to interpret visualization of this.
def get_default_model():
  input_image = Input(shape=TARGET_SIZE+(1,))

  x = BatchNormalization()(input_image)
  x = Conv2D(filters=64, kernel_size=(3,3), activation='relu')(x)
  x = BatchNormalization()(x)
  x = Conv2D(filters=64, kernel_size=(3,3), activation='relu', strides=(2,2), name='EarlyConv')(x)

  x = BatchNormalization()(x)
  x = Conv2D(filters=96, kernel_size=(3,3), activation='relu')(x)
  x = BatchNormalization()(x)
  x = Conv2D(filters=96, kernel_size=(3,3), activation='relu', strides=(2,2))(x)

  x = BatchNormalization()(x)
  x = Conv2D(filters=96, kernel_size=(3,3), activation='relu', name='MiddleConv')(x)
  x = BatchNormalization()(x)
  x = Conv2D(filters=96, kernel_size=(3,3), activation='relu', strides=(2,2))(x)

  x = BatchNormalization()(x)
  x = Conv2D(filters=128, kernel_size=(3,3), activation='relu')(x)
  x = BatchNormalization()(x)
  x = Conv2D(filters=128, kernel_size=(3,3), activation='relu', name='LastConv')(x)

  x = Flatten()(x)
  x = BatchNormalization()(x)
  x = Dense(512, activation='relu')(x)
  x = Dropout(rate=0.25)(x)

  preds = Dense(2,activation='softmax')(x) #final layer with softmax activation

  return Model(inputs=input_image,outputs=preds)

In [0]:
#@title Simple model definition
def get_simple_model():
  input_image = Input(shape=TARGET_SIZE+(1,))

  x = Conv2D(filters=16, kernel_size=(7,7), activation='relu', name='EarlyConv')(input_image)
  x = Conv2D(filters=32, kernel_size=(5,5), activation='relu')(x)
  x = Conv2D(filters=64, kernel_size=(5,5), activation='relu')(x)
  x = Conv2D(filters=128, kernel_size=(3,3), activation='relu', name='LastConv')(x)

  x = GlobalAveragePooling2D()(x)
  x = BatchNormalization()(x)
  x = Dense(512, activation='relu')(x)
  x = Dropout(rate=0.25)(x)

  preds = Dense(2,activation='softmax')(x) #final layer with softmax activation

  return Model(inputs=input_image,outputs=preds)

In [0]:
model = get_default_model()
#model = get_simple_model()
model.summary()
radam = RAdamOptimizer(learning_rate=1e-3)
model.compile(optimizer=radam, loss='categorical_crossentropy', metrics = ['accuracy'])

In [0]:
#@title Create the callbacks for visualization
#@markdown To provide some illustration, several callbacks are instantiated. Not all are used below, though.

#@markdown Double-click the header row to expand this cell and inspect the definitions. 

x_val_g = val_generator.next()
x_val_img = np.array(x_val_g[0])
x_val_lbl = np.array(x_val_g[1])

val_class_zero = (np.array([
    el for el, label in zip(x_val_img, x_val_lbl)
    if np.all(label == np.array([1] + [0]))
][0:9]), None)
val_class_one = (np.array([
    el for el, label in zip(x_val_img, x_val_lbl)
    if np.all(label == np.array([0] + [1]))
][0:9]), None)

#@markdown Which layer to visualize in __GradCam__:
LAYER = 'MiddleConv' #@param  ['MiddleConv', 'EarlyConv', 'LastConv']

#@markdown __Occlusion Sensitivity__ patch size:
PATCH_SIZE = 8 #@param {'type':'integer'}

callbacks_CAM = [
                 tfx.callbacks.GradCAMCallback(val_class_zero, layer_name='MiddleConv', class_index=0, output_dir='logs/PPMI/GradCam/LastConv/classPD-explPD'),
                 tfx.callbacks.GradCAMCallback(val_class_zero, layer_name='MiddleConv', class_index=1, output_dir='logs/PPMI/GradCam/LastConv/classPD-explHC'),
                 tfx.callbacks.GradCAMCallback(val_class_one, layer_name='MiddleConv', class_index=0, output_dir='logs/PPMI/GradCam/MiddleConv/classHC-explPD'),
                 tfx.callbacks.GradCAMCallback(val_class_one, layer_name='MiddleConv', class_index=1, output_dir='logs/PPMI/GradCam/MiddleConv/classHC-explHC')
]
callbacks_Occ = [
                 tfx.callbacks.OcclusionSensitivityCallback(val_class_zero,class_index=0, patch_size=PATCH_SIZE, output_dir='logs/PPMI/Occlusion/classPD-explPD'),
                 tfx.callbacks.OcclusionSensitivityCallback(val_class_zero,class_index=1, patch_size=PATCH_SIZE, output_dir='logs/PPMI/Occlusion/classPD-explHC'),
                 tfx.callbacks.OcclusionSensitivityCallback(val_class_one,class_index=0, patch_size=PATCH_SIZE, output_dir='logs/PPMI/Occlusion/classHC-explPD'),
                 tfx.callbacks.OcclusionSensitivityCallback(val_class_one,class_index=1, patch_size=PATCH_SIZE, output_dir='logs/PPMI/Occlusion/classHC-explHC')
]

tf_cb = tf.keras.callbacks.TensorBoard(histogram_freq=5, log_dir='logs/PPMI/metrics')

In [0]:
# train the model on the new data for a few epochs. Use the callbacks only afterwards to speed up the process.
history = model.fit_generator(generator=train_generator,
                              steps_per_epoch=train_generator.n//train_generator.batch_size,
                              epochs=50,
                             validation_data=val_generator,
                             validation_steps=val_generator.n//val_generator.batch_size,
                             verbose=0,
                             callbacks=[tf_cb])

In [0]:
# After that, only train two epochs to generate the visualizations. This is costly!
# Look into the embedded TensorBoard above to see results.
history = model.fit_generator(generator=train_generator,
                              steps_per_epoch=train_generator.n//train_generator.batch_size,
                              epochs=10,
                             validation_data=val_generator,
                             validation_steps=val_generator.n//val_generator.batch_size,
                             verbose=2,
                             callbacks=callbacks_CAM)

In [0]:
%load_ext tensorboard

In [0]:
import datetime, os

logs_base_dir = "./logs"
os.makedirs(logs_base_dir, exist_ok=True)
%tensorboard --logdir {logs_base_dir}

## Use `tf_explain` core to explain predictions directly

In [0]:
from tf_explain.core.smoothgrad import SmoothGrad
from tf_explain.core.grad_cam import GradCAM
from tf_explain.core.occlusion_sensitivity import OcclusionSensitivity
import matplotlib.pyplot as plt
%matplotlib inline

explainer = SmoothGrad()
gridSmoothGrad = explainer.explain(val_class_zero, model, class_index=0, num_samples=20, noise=1.)

explainer = GradCAM()
gridGradCam = explainer.explain(validation_data=val_class_zero, model=model, layer_name='MiddleConv', class_index=0)

explainer = OcclusionSensitivity()
gridOccl = explainer.explain(val_class_zero, model, class_index=0, patch_size=4)

plt.figure(figsize=(8, 8))
plt.imshow(gridSmoothGrad)
plt.show()
plt.figure(figsize=(8, 8))
plt.imshow(gridGradCam)
plt.show()
plt.figure(figsize=(8, 8))
plt.imshow(gridOccl)
plt.show()

# A toy example from the `tf_explain` authors.

The original code by the tf_explain authors, taken from https://github.com/sicara/tf-explain/blob/master/examples/callbacks/mnist.py

The original code does not cast the input data to float. This may cause a crash depending on TF version.

In [0]:
import numpy as np
import tensorflow as tf
import tf_explain

INPUT_SHAPE = (28, 28, 1)
NUM_CLASSES = 10

AVAILABLE_DATASETS = {
    'mnist': tf.keras.datasets.mnist,
    'fashion_mnist': tf.keras.datasets.fashion_mnist,
}
DATASET_NAME = 'fashion_mnist'  # Choose between "mnist" and "fashion_mnist"

# Load dataset
dataset = AVAILABLE_DATASETS[DATASET_NAME]
(train_images, train_labels), (test_images, test_labels) = dataset.load_data()
train_images = train_images.astype(np.float32)
test_images = test_images.astype(np.float32)

# Convert from (28, 28) images to (28, 28, 1)
train_images = train_images[..., tf.newaxis]
test_images = test_images[..., tf.newaxis]

# One hot encore labels 0, 1, .., 9 to [0, 0, .., 1, 0, 0]
train_labels = tf.keras.utils.to_categorical(train_labels, num_classes=NUM_CLASSES)
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes=NUM_CLASSES)

# Small function to get some instances of one class for testing (0-indexed)
def get_validation_instances(class_idx, num_instances):
  return (np.array([
      el for el, label in zip(test_images, test_labels)
      if np.all(label == np.array(class_idx * [0] + [1] + (NUM_CLASSES - class_idx - 1) * [0]))
  ][0:num_instances]), None)

def demo_model(x_train, y_train, x_val, y_val, params):
  # Create model
  img_input = tf.keras.Input(INPUT_SHAPE)

  x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(img_input)
  x = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', name='target_layer')(x)
  x = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(x)

  x = tf.keras.layers.Dropout(0.25)(x)
  x = tf.keras.layers.Flatten()(x)

  x = tf.keras.layers.Dense(128, activation='relu')(x)
  x = tf.keras.layers.Dropout(0.5)(x)

  x = tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')(x)

  model = tf.keras.Model(img_input, x)
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

  # Get some validation images    
  validation_class_zero = get_validation_instances(class_idx=0, num_instances=9)
  validation_class_eight = get_validation_instances(class_idx=7, num_instances=9)
  validation_class_three = get_validation_instances(class_idx=3, num_instances=9)
  validation_class_nine = get_validation_instances(class_idx=9, num_instances=9)

  # Instantiate callbacks
  # class_index value should match the validation_data selected above
  callbacks_mnist_GradCAM = [
      tf_explain.callbacks.GradCAMCallback(validation_class_three, 'target_layer', class_index=3, output_dir='logs/GradCam/class3-expl3'),
      tf_explain.callbacks.GradCAMCallback(validation_class_three, 'target_layer', class_index=9, output_dir='logs/GradCam/class3-expl9'),
      tf_explain.callbacks.GradCAMCallback(validation_class_nine, 'target_layer', class_index=3, output_dir='logs/GradCam/class9-expl3'),
      tf_explain.callbacks.GradCAMCallback(validation_class_nine, 'target_layer', class_index=9, output_dir='logs/GradCam/class9-expl9'),
  ]
  
  callbacks_mnist_Occl = [
      tf_explain.callbacks.OcclusionSensitivityCallback(validation_class_three, class_index=3, patch_size=params['patch_size'], output_dir='logs/MNIST/Occl/class3-expl3/patch'+str(params['patch_size'])),
      tf_explain.callbacks.OcclusionSensitivityCallback(validation_class_three, class_index=9, patch_size=params['patch_size'], output_dir='logs/MNIST/Occl/class3-expl9/patch'+str(params['patch_size'])),
      tf_explain.callbacks.OcclusionSensitivityCallback(validation_class_nine, class_index=3, patch_size=params['patch_size'], output_dir='logs/MNIST/Occl/class9-expl3/patch'+str(params['patch_size'])),
      tf_explain.callbacks.OcclusionSensitivityCallback(validation_class_nine, class_index=9, patch_size=params['patch_size'], output_dir='logs/MNIST/Occl/class9-expl9/patch'+str(params['patch_size'])),
  ]
  
  callbacks_fashion_mnist_Occl = [
      tf_explain.callbacks.OcclusionSensitivityCallback(validation_class_eight, class_index=7, patch_size=params['patch_size'], output_dir='logs/F-MNIST/Occl/class7-expl7/patch'+str(params['patch_size'])),
      tf_explain.callbacks.OcclusionSensitivityCallback(validation_class_eight, class_index=9, patch_size=params['patch_size'], output_dir='logs/F-MNIST/Occl/class7-expl9/patch'+str(params['patch_size'])),
      tf_explain.callbacks.OcclusionSensitivityCallback(validation_class_nine, class_index=3, patch_size=params['patch_size'], output_dir='logs/F-MNIST/Occl/class9-expl7/patch'+str(params['patch_size'])),
      tf_explain.callbacks.OcclusionSensitivityCallback(validation_class_nine, class_index=9, patch_size=params['patch_size'], output_dir='logs/F-MNIST/Occl/class9-expl9/patch'+str(params['patch_size'])),
  ]
  # Start training
  out = model.fit(x_train, y_train, epochs=15, callbacks=callbacks_fashion_mnist)
  return out, model


## Use Talos for hyperparameter tuning

We can 'misuse' the talos package which is originally intended to ease hyperparameter tuning to run a batch of experiments, in this case changing the size of the occlusion patch between 4 and 12 in increments of 4.

In [0]:
p = {
    'patch_size': [4, 6, 8, 10]
}

In [0]:
h = talos.Scan(train_images.astype(np.float32), train_labels, p, demo_model, 'patchsize')

In [0]:
!zip -r /content/logs.zip /content/logs

In [0]:
from google.colab import files
files.download("/content/logs.zip")